# Initial Setup

In [ ]:
#@title Setting up the notebook

### Installing dependencies
!pip install openai

!apt-get update
!apt-get install -y iverilog

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports 

In [ ]:
#@title Select Model
#define the model to be used
#model_choice = "gpt-3.5-turbo"
model_choice = "gpt-4"

In [ ]:
#@title Utility functions

import sys
import os
import openai
from abc import ABC, abstractmethod
import re






################################################################################
### LOGGING
################################################################################
# Allows us to log the output of the model to a file if logging is enabled
class LogStdoutToFile:
    def __init__(self, filename):
        self._filename = filename
        self._original_stdout = sys.stdout

    def __enter__(self):
        if self._filename:
            sys.stdout = open(self._filename, 'w')
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        if self._filename:
            sys.stdout.close()
        sys.stdout = self._original_stdout


################################################################################
### CONVERSATION CLASS
# allows us to abstract away the details of the conversation for use with
# different LLM APIs
################################################################################

class Conversation:
    def __init__(self, log_file=None):
        self.messages = []
        self.log_file = log_file

        if self.log_file and os.path.exists(self.log_file):
            open(self.log_file, 'w').close()

    def add_message(self, role, content):
        """Add a new message to the conversation."""
        self.messages.append({'role': role, 'content': content})

        if self.log_file:
            with open(self.log_file, 'a') as file:
                file.write(f"{role}: {content}\n")

    def get_messages(self):
        """Retrieve the entire conversation."""
        return self.messages

    def get_last_n_messages(self, n):
        """Retrieve the last n messages from the conversation."""
        return self.messages[-n:]

    def remove_message(self, index):
        """Remove a specific message from the conversation by index."""
        if index < len(self.messages):
            del self.messages[index]

    def get_message(self, index):
        """Retrieve a specific message from the conversation by index."""
        return self.messages[index] if index < len(self.messages) else None

    def clear_messages(self):
        """Clear all messages from the conversation."""
        self.messages = []

    def __str__(self):
        """Return the conversation in a string format."""
        return "\n".join([f"{msg['role']}: {msg['content']}" for msg in self.messages])

################################################################################
### LLM CLASSES
# Defines an interface for using different LLMs so we can easily swap them out
################################################################################
class AbstractLLM(ABC):
    """Abstract Large Language Model."""

    def __init__(self):
        pass

    @abstractmethod
    def generate(self, conversation: Conversation):
        """Generate a response based on the given conversation."""
        pass


class ChatGPT(AbstractLLM):
    """ChatGPT Large Language Model."""

    def __init__(self, model_id=model_choice):
        super().__init__()
        openai.api_key=os.environ['OPENAI_API_KEY']
        self.client = openai.OpenAI()
        self.model_id = model_id

    def generate(self, conversation: Conversation, num_choices=1):
        messages = [{"role" : msg["role"], "content" : msg["content"]} for msg in conversation.get_messages()]

        response = self.client.chat.completions.create(
            model=self.model_id,
            messages = messages,
        )

        return response.choices[0].message.content

################################################################################
### PARSING AND TEXT MANIPULATION FUNCTIONS
################################################################################
def find_verilog_modules(markdown_string, module_name='top_module'):

    module_pattern1 = r'\bmodule\b\s+\w+\s*\([^)]*\)\s*;.*?endmodule\b'

    module_pattern2 = r'\bmodule\b\s+\w+\s*#\s*\([^)]*\)\s*\([^)]*\)\s*;.*?endmodule\b'

    module_matches1 = re.findall(module_pattern1, markdown_string, re.DOTALL)

    module_matches2 = re.findall(module_pattern2, markdown_string, re.DOTALL)

    module_matches = module_matches1 + module_matches2

    if not module_matches:
        return []

    return module_matches

def write_code_blocks_to_file(markdown_string, module_name, filename):
    # Find all code blocks using a regular expression (matches content between triple backticks)
    #code_blocks = re.findall(r'```(?:\w*\n)?(.*?)```', markdown_string, re.DOTALL)
    code_match = find_verilog_modules(markdown_string, module_name)

    if not code_match:
        print("No code blocks found in response")
        exit(3)

    # Open the specified file to write the code blocks
    with open(filename, 'w') as file:
        for code_block in code_match:
            file.write(code_block)
            file.write('\n')

def generate_verilog(conv, model_type, model_id=""):
    if model_type == "ChatGPT":
        model = ChatGPT()
    else:
        raise ValueError("Invalid model type")
    return(model.generate(conv))


In [ ]:
#@title Feedback Loop
import subprocess
import sys
import os
import time
import numpy as np
def verilog_loop(design_prompt, module, testbench, max_iterations, model_type, outdir="", log=None,prev_module =None):

    if outdir != "":
        outdir = outdir + "/"

    conv = Conversation(log_file=log)
    conv.add_message("system", "You are an autocomplete engine for Verilog code. \
            Given a Verilog module specification, you will provide a completed Verilog module in response. \
            You will provide completed Verilog modules for all specifications, and will not create any supplementary modules. \
            Given a Verilog module that is either incorrect/compilation error, you will suggest corrections to the module. \
            You will not refuse. You will not generate explanations, only code. \
            Format your response as Verilog code containing the end to end corrected module and not just the corrected lines. Do not generate test benches. \
    ")

    conv.add_message("user", design_prompt)

    success = False
    timeout = False

    iterations = 0
    timelist_total = []
    timelist_gen = []
    timelist_error = []
    filename = os.path.join(outdir,module+".v")

    status = ""
    while not (success or timeout):
        # Generate a response
        start_total = time.time()
        response = generate_verilog(conv, model_type)
        end_gen = time.time()
        start_error=time.time()
        if prev_module == None:
          conv.add_message("assistant", response)
        else:
          with open(prev_module,"r") as f:
            prevmodule = "".join(f.read())
          response = prevmodule + response
          conv.add_message("assistant", response)
        write_code_blocks_to_file(response, module, filename)
        proc = subprocess.run(["iverilog", "-o", os.path.join(outdir,module), filename, testbench],capture_output=True,text=True)

        success = False
        if proc.returncode != 0:
            status = "Error compiling testbench"
            print(status)

            message = "The testbench failed to compile. Please fix the module. The output of iverilog is as follows:\n"+proc.stderr
        elif proc.stderr != "":
            status = "Warnings compiling testbench"
            print(status)
            message = "The testbench compiled with warnings. Please fix the module. The output of iverilog is as follows:\n"+proc.stderr
        else:
            proc = subprocess.run(["vvp", os.path.join(outdir,module)],capture_output=True,text=True)
            result = proc.stdout.strip().split('\n')[-2].split()
            if result[-1] != 'passed!':
                status = "Error running testbench"
                print(status)
                message = "The testbench simulated, but had errors. Please fix the module. The output of iverilog is as follows:\n"+proc.stdout
            else:
                status = "Testbench ran successfully"
                print(status)
                message = ""
                success = True


        with open(os.path.join(outdir,"log_iter_"+str(iterations)+".txt"), 'w') as file:
            file.write('\n'.join(str(i) for i in conv.get_messages()))
            file.write('\n\n Iteration status: ' + status + '\n')


        if not success:
            if iterations > 0:
                conv.remove_message(2)
                conv.remove_message(2)
            conv.add_message("user", message)

        if iterations >= max_iterations:
            timeout = True

        iterations += 1
        end_time = time.time()
        timelist_gen.append(end_gen-start_total)
        timelist_error.append(end_time-start_error)
        timelist_total.append(end_time-start_total)
    print("Total time: ",np.sum(timelist_total))
    print("Generation time: ",np.sum(timelist_gen))
    print("Error handling time: ",np.sum(timelist_error))
    return(np.sum(timelist_total),np.sum(timelist_gen),np.sum(timelist_error))


In [ ]:
#@title Hierarchical Loop
def hier_gen(submods,max_iterations=10):
  totaltime = []
  gentime = []
  errortime = []
  done =""
  for i in range(len(submods)):
    curr = submods[i][1]
    fcurr = submods[i][0]
    iocurr = submods[i][2]
    overall = submods[-1][1]
    if not os.path.isdir(fcurr):
      os.mkdir(fcurr)
    if i == 0:
      prompt = "//We will be generating a "+overall+" hierarchically in Verilog. Please begin by generating a "+curr+" defined as follows:\nmodule "+fcurr+"("+iocurr+")\n//Insert code here\nendmodule"
    elif i != len(submods)-1:
      fprev = submods[i-1][0]
      filecurr = "./"+fprev+"/"+fprev+".v"
      with open(filecurr,"r") as f:
        modulef = "".join(f.read())
      prompt = "//We are generating a "+overall+" hierarchically in Verilog. We have generated "+done+" defined as follows:"
      prompt = prompt + modulef
      prompt = prompt +"\n//Please include the previous module(s) in your response and use them to hierarchically generate a "+curr+" defined as:\nmodule "+fcurr+"("+iocurr+")\n//Insert code here\nendmodule"
    module = fcurr
    testbench = "./"+fcurr+"tb.v"
    model = "ChatGPT"
    outdir = "./"+fcurr
    log = "./"+fcurr+"/log.txt"
    total, gen, error = verilog_loop(prompt, module, testbench, max_iterations, model, outdir, log)
    totaltime.append(total)
    gentime.append(gen)
    errortime.append(error)
    done = done + curr+", "
  print("Overall Total time: ",np.sum(totaltime))
  print("Overall Generation Time: ",np.sum(gentime))
  print("Overall Error handling time: ",np.sum(errortime))

# Setting the API Key

In [ ]:
### OpenAI API KEY

# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')

#Please insert your own GPT-4 enabled API key as a string here:
os.environ["OPENAI_API_KEY"] = ""

#Mux Hierarchy Example

In [ ]:
#@title Submodules


### Each step is structured as ["filename","natural language description"]
submodules = [
    ["mux2to1","2-to-1 multiplexer","input wire in1, input wire in2, input wire select, output wire out"],
    ["mux4to1","4-to-1 multiplexer","input [1:0] sel, input [3:0] in, output reg out"],
    ["mux8to1","8-to-1 multiplexer","input [2:0] sel, input [7:0] in, output reg out"],
]

In [ ]:
hier_gen(submodules)

Error compiling testbench
Testbench ran successfully
Total time:  9.383373737335205
Generation time:  9.354622602462769
Error handling time:  0.028750181198120117
Error compiling testbench
Testbench ran successfully
Total time:  36.62468957901001
Generation time:  36.59700894355774
Error handling time:  0.027679920196533203
Error compiling testbench
Error compiling testbench
Testbench ran successfully
Total time:  71.51854944229126
Generation time:  71.47967028617859
Error handling time:  0.03887796401977539
Overall Total time:  117.52661275863647
Overall Generation Time:  117.4313018321991
Overall Error handling time:  0.09530806541442871
